## Comp3710 Final Project

## Shor's Algorithm kata using Qsharp environment

## Abstract
This is a Qsharp tutorial on Shor's algorithm that aims to teach you each step to construct the algorithm. The shor's algorithm contains classic arthmetic part that could be utilized by using other programming language like python or cpp. For consistency purpose, this kata will purely use Qsharp. Due to limit resource and some weakness of Qsharp, some part in this tutorial can be improved and polished.

### The flowchart of shor's!
![flowchart](./image/flowchart.jpg)

In [1]:
// import relevant namespace
open Microsoft.Quantum.Arithmetic;
open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Math;
open Microsoft.Quantum.Random;
open Microsoft.Quantum.Convert;
open Microsoft.Quantum.Oracles;
open Microsoft.Quantum.Characterization;
open Microsoft.Quantum.Measurement;

We will use LittleEndian throughout

In [2]:
%config dump.basisStateLabelingConvention="LittleEndian"
%config dump.truncateSmallAmplitudes = "true"

"true"

### The quantum circuit
![circuit](./image/shorcircuit.jpg)

### Luckily, we got all the numerical operation inbuilt in Qsharp library that operates on the superposition arithmetic on qubits, let's get familiar with these operations.
1. ModularMultiplyByConstantLE
2. PowerMod
3. ExpModI
4. BitsizeI
4. GreatestCommonDivisorI
5. ApplyXorInplace
6. MultiplyByModularInteger
7. RobustPhaseEstimation
8. PhaseEstimationImpl

### Naming conventions: a refers to the generator, x or r refers to the power and N refers to the number to be factorized.


### Step1: verify number $N$ is a non-prime number and find the generator $a$ that is coprime to $N$

In [3]:
// built-in function IsCoprimeI is helpful in determining whether two integer are coprime

function determ_prime(N : Int ): Bool {
    for divisor in (2 .. N-1){
        if (N%divisor == 0) {
            Message($"Checked, {N} is non-prime");
            return false;}
        }
    Message($"The number {N} is a prime, pick another N");
    return true;
}

function determ_coprime(generator : Int, N : Int ): Bool {
        if (not IsCoprimeI(generator, N)){
        Message("The generator is not coprime with N, pick another generator");
        }
        return IsCoprimeI(generator, N);
}

function determ_precision(N :Int):Int{
    return 2*BitSizeI(N) + 1;
}

function calculate_remainder(a :Int, r:Int, N :Int):Int{
    return ExpModI(a,r,N);
}

determ_prime, determ_coprime, determ_precision, calculate_remainder

### Step2: prepare circuit

#### 2.1 Prepare superposition with two input integer
Demo purpose. Does not has practical usage.
It shows how to use a control qubit to encode two integer into register. The two integer are N and a.

In [4]:
operation superposition(integer1 :Int, integer2 :Int, register : LittleEndian): Unit is Adj + Ctl {
    use control_bit = Qubit() {
        H(control_bit);
        within {X(control_bit);} 
        apply {Controlled ApplyXorInPlace([control_bit],(integer1, register));}
        Controlled ApplyXorInPlace([control_bit],(integer2, register));
        (ControlledOnInt(integer2, Y))(register!, control_bit);
    }
}

superposition

In [5]:
operation Unitary_Mod(integer1 :Int, integer2 :Int,multiplier : Int,modulus : Int): () {
    // we want to keep track of remainder, the remainder is less than modulus
    // the maximum #bits used to encode remainder
    let encoding_bits = BitSizeI(modulus);
    use target = Qubit[encoding_bits] {
        let register = LittleEndian(target);
        // prepare the superposition w.r.t two integer
        superposition(integer1,integer2, register);
        MultiplyByModularInteger(multiplier, modulus, register);
    }
}


Warning QS3002: 
File: /snippet_.qs 
Position: [ln 1, cn 85] 
Deprecated syntax. Use the type name "Unit" instead.


Unitary_Mod

#### 2.2 construct inverse QFT
Implementation: Apply the H gate to create superposition and then R1Frac gate apply the phase $e^{i\pi n/2^{k}}$. In the end, swap the binaryFraction into reverse order. It is necessary to build QFT from scratch since the inbuilt QFT function use BigEddian as default.
For details, please refer to the QFT kata tutorial.

In [6]:
operation ReverseRegister (register : Qubit[]) : Unit is Adj+Ctl {
    let N = Length(register);
    for ind in 0 .. N / 2 - 1 {
        SWAP(register[ind], register[N - 1 - ind]);
    }
}

ReverseRegister

In [7]:
operation QFT(registers:Qubit[]): Unit is Adj+Ctl{
    let length = Length(registers);
    for i in 0 .. length-1{
        H(registers[i]);
        for k in 1 .. i -1{
        Controlled R1Frac([registers[k]],(2, i-k+1, registers[k]));
        }
    }
    ReverseRegister(registers);
}

operation inverse_QFT(registers:Qubit[]): Unit is Adj+Ctl{
    Adjoint QFT(registers);
}

QFT, inverse_QFT

### Step3: prepare modular oracle

In [8]:
operation modularOracle(a : Int, N : Int, r : Int, regist : Qubit[]): Unit is Adj + Ctl {
    let b = determ_coprime(a, N);
    
    // perform modular operation on target register
    // ExpModI gives a^r (mod N)
    // MultiplyByModularInteger gives | a^r (mod N) * register>
    MultiplyByModularInteger(ExpModI(a, r, N),N, LittleEndian(regist));
}

modularOracle

### Step4: estimate order from the modular function 
#### 4.1.1 estimate frequency from the modular oracle use inbuilt function
RobustPhaseEstimation is very useful here or it can be replaced by our own QPE ConstructPhaseEstimation below.

In [9]:
// assuming the operation takes in qubits_size, oracle and returns the phase e^theta*Fi*2^(n-1)

operation GetFrequency(Oracle : ((Int, Qubit[]) => Unit is Adj+Ctl), bit_size : Int): Int {
    use regist = Qubit[bit_size] {
        let precision = 2*(bit_size)+1;
        
        // apply XOR
        ApplyXorInPlace(1, LittleEndian(regist));
        
        // inbuilt RobustPhaseEstimation performs non-iterative estimation
        let phase = RobustPhaseEstimation(precision, DiscreteOracle(Oracle), (LittleEndian(regist))!);
        ResetAll(regist);
        
        return Round((phase * IntAsDouble(2 ^ (precision-1))) / PI());
    }
}

GetFrequency

#### 4.1.2 estimate frequency from constructed modular oracle
Build our own phase estimation
For details, please refer to the QPE kata

In [10]:
operation QPE_Reference (a : Int, N : Int, regs : Qubit[], eigen : Qubit[]) : ()
{
        // create superpositions
        for (i in 0..(Length(regs) - 1)) {
            H(regs[i]);
        }
        
        // the phase is e^(i*theta*phi2^(n-1))
        // const to record the 2^(n-1)
        mutable power = 1;
        for (j in 0..(Length(regs) - 1)) {
            (Controlled modularOracle) ([regs[Length(regs) -j-1]], (a, N, power, eigen)); 
            set power = power * 2;

        (inverse_QFT)(regs);

    }
    
}


Warning QS3002: 
File: /snippet_.qs 
Position: [ln 1, cn 79] 
Deprecated syntax. Use the type name "Unit" instead.

Warning QS3306: 
File: /snippet_.qs 
Position: [ln 4, cn 13] 
Deprecated syntax. Parentheses here are no longer required and will not be supported in the future.

Warning QS3306: 
File: /snippet_.qs 
Position: [ln 11, cn 13] 
Deprecated syntax. Parentheses here are no longer required and will not be supported in the future.


QPE_Reference

In [11]:
operation ConstructPhaseEstimation(x:Int, N:Int, prec : Int): Int{
    mutable answ = 0;
    use eigen = Qubit[BitSizeI(N)] {
        X(eigen[0]);
        use regist = Qubit[prec] {
            QPE_Reference(x,N,regist,eigen);
            
            for i in 0 .. prec-1{
                set answ = MResetX(regist[i]) == One ? answ *2+1 | answ *2;
            }
        }
        }
        return answ;
    

}

ConstructPhaseEstimation

#### 4.2 repeat finding the frequency till it is valid
Due the huge range of possible periods, use the trick of continued fraction expension. It will result in uncerity of finding a period.

In [12]:
operation GetPeriod(generator : Int, N : Int): Int {

    let b = determ_coprime(generator,  N);
 
    let bitSize = BitSizeI(N);
    let measurement_precision = determ_precision(N);
    mutable res = 1;
    mutable frequency = 0;
 
    // perform iterative period finding until the reminader is 1
    repeat {
        // frequency is get from QPE of modular oracle
        set frequency = GetFrequency(modularOracle(generator,  N, _, _), bitSize);
        
        // continued fraction expension
        if (frequency != 0) {
            // compute frequency/precision
            let frac = Fraction(frequency, 2^measurement_precision);
            // find the continued fraction convergent close to N
            let fraction =  ContinuedFractionConvergentI(frac,  N);
            // Fraction = (Fst,Snd) = (Numerator, Denominator) 
            let denominator = AbsI(Snd(fraction!));
            set res = (denominator * res) / GreatestCommonDivisorI(res, denominator);
        } else {
            Message("The frequency was 0 from fraction calculation");
            Message("run another frequency estimation");
        }
    } until(ExpModI(generator, res,  N) == 1);

    return res;
}

GetPeriod

### Step 5 Assemble everything

In [13]:
// qsharp has inbuilt GreatestCommonDivisor "GreatestCommonDivisorI" and "GreatestCommonDivisorL"
// use GreatestCommonDivisorI for smaller int number as our gcd
// use gcd to update our guess given period 
// r is our period here, also the power

operation update_guess(a :Int, r : Int, N: Int):(Bool,Int) {
    mutable new_guess = 1;
    if (r % 2 == 0){
        let update_generator = ExpModI(a, r/2, N);
        if (update_generator != N -1 or update_generator != N +1){

            // initialize a uniform distribution on selecting new guess
            let distribution = DiscreteUniformDistribution(0,1);
       
            let guess1 = GreatestCommonDivisorI(update_generator-1, N);
            let guess2 = GreatestCommonDivisorI(update_generator+1, N);
            let index = distribution::Sample();
            
            // generate new guess
            if (index == 0){
               set new_guess = guess1;}
            else { set new_guess = guess2;}
            
            // get rid of guess equals one or N itself
            if (new_guess != 1 and new_guess != N){
                return (true, new_guess);} 
            else { return(false, new_guess); }
        } else { return(false,new_guess); }
    } else {
        return (false, new_guess);
        }
}

update_guess

In [14]:
operation Factorization(number : Int): Int {
    
    // if it is prime, the factor is 1 and itself
    if ( determ_prime(number)) {
        return 1;
    }
    
    // initalize the two factors to be 1
    mutable first_factor = 1;
    mutable sec_factor = 1;
    
    // the condition varible to terminate the loop
    mutable success = false;
    Message($"Factoring {number} in process");
    repeat {
        let generator = DrawRandomInt(2, number - 2);
        Message($"The random chosen generator is {generator}");
        if (IsCoprimeI(generator, number)) {
           
            Message($"Finding the period...");
            let period = GetPeriod(generator, number);
            Message($"The period is {period}");
            set (success, first_factor) = update_guess(generator, period, number);
            set sec_factor = number / first_factor;
            if(success){
            Message($"Successed, the new guesses is {first_factor}");}
            else{Message($"Failed, execute another period finding");}
        } else {
            let common_factor = GreatestCommonDivisorI(number, generator);
            Message($"Luckily, the random chosen generator {generator} contains co-factor");
            
            if (common_factor!= 1 or common_factor != number){
            set first_factor = common_factor;
            set sec_factor = number / common_factor;} 
            else{
            set first_factor = generator;
            }
            set success = true;}
    } until(success);
    Message($"The factors are {first_factor} and {sec_factor}");
    return first_factor;
}

Factorization

### Testing

#### Single Testing

In [15]:
operation single_test() : (Int, Int) {
            let N = 15;
            let guess = Factorization(N);
            return (guess,N/guess);
        }

single_test

In [16]:
%simulate single_test

Checked, 15 is non-prime
Factoring 15 in process
The random chosen generator is 6
Luckily, the random chosen generator 6 contains co-factor
The factors are 3 and 5


(3, 5)

In [17]:
operation single_test2() : (Int, Int) {
            let N = 25;
            let guess = Factorization(N);
            return (guess,N/guess);
        }

single_test2

In [18]:
%simulate single_test2

Checked, 25 is non-prime
Factoring 25 in process
The random chosen generator is 14
Finding the period...
The period is 10
Failed, execute another period finding
The random chosen generator is 13
Finding the period...
The period is 20
Failed, execute another period finding
The random chosen generator is 14
Finding the period...
The period is 10
Failed, execute another period finding
The random chosen generator is 16
Finding the period...
The frequency was 0 from fraction calculation
run another frequency estimation
The period is 5
Failed, execute another period finding
The random chosen generator is 17
Finding the period...
The period is 20
Failed, execute another period finding
The random chosen generator is 5
Luckily, the random chosen generator 5 contains co-factor
The factors are 5 and 5


(5, 5)

#### Group Testing

Qsharp is only a simulator rather than a actual QC. Do not try excessive large number, it may result in your computer stuck. It would take 15 mins or so to run through all the test.

In [19]:
operation group_test():Int{
            // case1
            let N1 = 21;
            let g1 = Factorization(N1);
            Message("\n");
            
             // case2
            let N2 = 15;
            let g2 = Factorization(N2);
            Message("\n");
            
             // case3
            let N3 = 12;
            let g3 = Factorization(N3);
            Message("\n");
            
             // case4
            let N4 = 9;
            let g4 = Factorization(N4);
            Message("\n");
            
             // case5
            let N5 = 99;
            let g5 = Factorization(N5);
            Message("\n");
            
            // for input larger than 100, it takes more qubits memory to finish
            // qsharp is only a simulator rather than a actual QC, it may not finish runing it
            
            // case6
            //let N6 = 189;
            //let g6 = Factorization(N6);
            //Message("\n");
            
            // case7
            //let N7 = 256;
            //let g7 = Factorization(N7);
            //Message("\n");
            
            // case8
            //let N8 = 475;
            //let g8 = Factorization(N8);
            //Message("\n");
            
            
            return 0;
        }

group_test

In [ ]:
%simulate group_test

Checked, 21 is non-prime
Factoring 21 in process
The random chosen generator is 11
Finding the period...
